In [179]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [180]:
words = open('names.txt', 'r').read().splitlines()
words[:8]  

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [181]:
len(words)

32033

In [182]:
chars = sorted(list(set(''.join(words))))
stoi = { ch:i+1 for i,ch in enumerate(chars) } # 1 is reserved for padding
stoi['.'] = 0
itos = { i:ch for ch,i in stoi.items() }
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [183]:
# Build Dataset
block_size = 3
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [209]:
def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [247]:
X, Y = Xtr, Ytr  # for training
print(f"X shape: {X.shape}, Y shape: {Y.shape}")


# Training
g = torch.Generator().manual_seed(2147483647)  # for reproducibility

embedding_dim = 10  # dimension of character embeddings
layer_size = 200
nchars = 27

# take character as input and get 2d embedding
C = torch.randn((nchars, embedding_dim), generator=g)  # character embeddings
# character embedding -> hidden layer
W1 = torch.randn((embedding_dim * block_size, layer_size), generator=g)  # input to hidden layer
b1 = torch.randn(layer_size, generator=g)  # bias for hidden layer
# hidden layer -> output layer
W2 = torch.randn((layer_size, nchars), generator=g)  # hidden to
b2 = torch.randn(nchars, generator=g)  # bias for output layer

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

X shape: torch.Size([182625, 3]), Y shape: torch.Size([182625])


In [248]:
lri = []
lossi = []
stepi = []

In [263]:
nsteps = 50000
lr = 0.01
stepi = []
lossi = []
for i in range(nsteps):
    # # sample a random batch of data
    ix = torch.randint(0, X.shape[0], (32,), generator=g)  # 32 random indices
    X_batch = X[ix]
    Y_batch = Y[ix]

    # forward pass
    emb = C[X_batch]
    h = torch.tanh(emb.view(emb.shape[0], -1) @ W1 + b1)
    logits = h @ W2 + b2

    # compute loss
    loss = F.cross_entropy(logits, Y_batch)
    # print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -lr * p.grad

    stepi.append(i)
    lossi.append(loss.log10().item())


In [256]:
# plt.plot(stepi, lossi)

In [265]:
# forward pass
# X, Y = Xdev, Ydev  # for validation
X, Y = Xtr, Ytr  # for validation
emb = C[X]
h = torch.tanh(emb.view(emb.shape[0], -1) @ W1 + b1)
logits = h @ W2 + b2

# compute loss
loss = F.cross_entropy(logits, Y)
print(loss.item())


2.268083095550537


In [266]:
# forward pass
X, Y = Xte, Yte  # test
emb = C[X]
h = torch.tanh(emb.view(emb.shape[0], -1) @ W1 + b1)
logits = h @ W2 + b2

# compute loss
loss = F.cross_entropy(logits, Y)
print(loss.item())

2.0757009983062744


In [267]:
g = torch.Generator().manual_seed(2147483647)  # for reproducibility

for _ in range(20):
    out = []
    context = [0] * block_size  # start with padding
    while True:
        emb = C[torch.tensor(context)]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        if ix == 0:  # stop if we hit the padding character
            break
        out.append(itos[ix])
        context = context[1:] + [ix]
    print(''.join(out))

dex
marialaurailah
tahammilimithainella
kaman
arreliceulliah
milian
moziellah
jazithda
kaley
masedly
tiaverry
folstofilie
avon
las
kasorvonneslyn
alaira
aler
dan
kez
myker
